# **Binomial Options Pricing Model - American Options**

This document will use the theory from the European Theory notebook except it will rely on the early exercise formula here. The functions will be made for both call and put options. 

## American Put Options Formulae

For American put options, at the terminal node layer i.e. $T = t_{N}$, the payoff for the option is given by 
$$ C_{N,j} = (K - S_{N,j})^{+}$$

However for all other nodes, this is where the early-exercise option features: <br>
For a given node at $(i,j)$, the payoff is given by the **max of the exercise value or the hold value**

$$C_{i,j} = max((K - S_{i,j})^+ , e^{-r \Delta t} \{ pC_{i+1,j+1} + (1-p)C_{i+1,j} \} $$


Notice how this is similar to the European formula except there is also the option between the discounted expected value (hold value) or the payoff if exercised at that node (time)

## American Call Options 

For this type of options, it is generally never optimal to exercise the option before expiry **ONLY IF** 
1. The underlying asset pays no dividends
2. The interest rate is non-negative

So it can simply act as a European Call Option and acn hence be converged to the Black-Scholes formula. The code below will calculate it using binomial trees and early-exercise trade-offs for consistency.

## **Parameter & Code Setup**

In [1]:
import numpy as np

In [2]:
# defining variables
#TODO: change u and d to account for volatility
S0 = 100    # initial stock price
K = 100     # strike price
T = 1       # time to maturity (years)
N = 3       # no. of time-steps
r = 0.06    # risk-free annual interest rate
u = 1.1     # up factor
d = 1/u     # down factor (inverse for recombining tree)
opt = 'P'   # options type (C - call ; P - put)

### American Binomial Tree - NumPy (Fast)

In [10]:
def american_binomial(S0,K,T,N,r,u,d,opt='P'):
    # first compute equation constants
    dt = T/N 
    p = (np.exp(r*dt) - d)/(u - d)
    discount = np.exp(-r * dt)
    
    # Stock prices at terminal layer
    S = S0 * u ** (np.arange(0,N+1,1)) * d ** (np.arange(N,-1,-1))

    if opt == 'P':
        C = np.maximum(K - S, np.zeros(N+1))
    else:
        C = np.maximum(S - K, np.zeros(N+1))
    
    # work through the tree backwards
    for i in range(N-1,-1,-1):
        S = S0 * u ** (np.arange(0,i+1,1)) * d ** (np.arange(i,-1,-1)) # calculate stock prices at layer
        #print(C)
        C[:i+1] = discount * ( p*C[1:i+2] + (1-p)*C[0:i+1] ) # calculate payoff for each node
        C = C[:-1] # knocks off last element to save memory

        if opt == 'P':
            C = np.maximum(C, K - S)
        else:
            C = np.maximum(C, S - K)

    return C[0]

P = round(american_binomial(S0,K,T,N,r,u,d,opt='P').item(),2)
C = round(american_binomial(S0,K,T,N,r,u,d,opt='C').item(),2)
print(f'Value of American Put: ${P}')
print(f'Value of American Call: ${C}')

Value of American Put: $4.65
Value of American Call: $10.15
